# Notebook with examples for Transformations

In [ ]:
import numpy as np
from tvsclib.mixed_system import MixedSystem
from tvsclib.strict_system import StrictSystem

from tvsclib.toeplitz_operator import ToeplitzOperator
from tvsclib.system_identification_svd import SystemIdentificationSVD
from tvsclib.transformations.reduction import Reduction
from tvsclib.transformations.input_normal import InputNormal
from tvsclib.transformations.output_normal import OutputNormal

from tvsclib.stage import Stage

import tvsclib.utils as utils
import matplotlib.pyplot as plt

## Create a system

Here we create a system. 
This time we weil create a tridiagonal matrix. After this we will invert the matrix.
By this we get an semiseperable matrix. Thsi mattrix has the property that all submatrices taken form the lower and upper triangular parts have rank 1.
This also means that the a state dim of 1 is sufficent.

In [ ]:
dims_in =  [1]*8
dims_out = [1]*8
matrix = np.random.rand(sum(dims_out), sum(dims_in))
matrix = matrix - np.tril(matrix,-2) - np.triu(matrix,2) # Banded shape
plt.figure()
plt.matshow(abs(matrix))

matrix = np.linalg.inv(matrix)                           # Obscure structure
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T)

plt.figure()
plt.matshow(abs(matrix))

system = MixedSystem(S)
system_causal = system.causal_system
system_anticausal = system.anticausal_system

print(f"Causal state dimensions: {system_causal.dims_state}")
print(f"Anticausal state dimensions: {system_anticausal.dims_state}")


## Create a more complex system

In [ ]:
dims_in =  [2, 1, 2, 1, 5, 2,10, 3, 2, 1, 3, 2, 4, 2, 5,20,30,10,10,10,15]
dims_out = [1, 2, 1, 2, 5, 2, 7, 3, 2, 1, 5, 7, 2, 1, 2,20,30,10,10,10,15]
matrix = np.random.rand(sum(dims_out), sum(dims_in))
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=1e-10)
system = MixedSystem(S)
system_causal = system.causal_system
system_anticausal = system.anticausal_system

The system is already mininmal and balnaced.

In [ ]:
print("Causal minimal:",system_causal.is_minimal())
print("Anticausal minimal:",system_anticausal.is_minimal())
print("Causal balanced:",system_causal.is_balanced(tolerance=1e-11))
print("Anticausal balanced:",system_anticausal.is_balanced(tolerance=1e-11))

## Input normal

We can now transform it to input normal form

In [ ]:
sys_causal_inp = InputNormal().apply(system_causal)
sys_anticausal_inp = InputNormal().apply(system_anticausal)
print("Causal input normal:",sys_causal_inp.is_input_normal())
print("Anticausal input normal:",sys_anticausal_inp.is_input_normal())

Now check if the systems systems are really equivalent

In [ ]:
print(np.max(np.abs(sys_causal_inp.to_matrix()-system_causal.to_matrix())))
print(np.max(np.abs(sys_anticausal_inp.to_matrix()-system_anticausal.to_matrix())))

Input normality means that the collumns in the observability matrix are orthonormal.
This mans that the gramian is the identity matrix.

Lets illustrate this here:

In [ ]:
n = 2
gram = sys_anticausal_inp.reachability_matrix(n)@sys_anticausal_inp.reachability_matrix(n).T
print(gram)
np.allclose(gram,np.eye(gram.shape[0]))

Illustrate that the systems got copied

In [ ]:
sys_anticausal_inp.stages is system_anticausal.stages

## Output normal and Input Normal

In [ ]:
sys_causal_out = OutputNormal().apply(system_causal)
sys_anticausal_out = OutputNormal().apply(system_anticausal)
print("Causal output normal:",sys_causal_out.is_output_normal())
print("Anticausal output normal:",sys_anticausal_out.is_output_normal())

Output normality means that the rows in the observability matrix are orthonormal.
This mans that the gramian is the identity matrix.

Lets illustrate this here:

In [ ]:
n = 2
gram = sys_anticausal_out.observability_matrix(n)@sys_anticausal_out.observability_matrix(n).T
print(gram)
np.allclose(gram,np.eye(gram.shape[0]))

## Reduction

First create a nonminimal system.
For this we set try to represent a low rank 1 matrix and set the epsilon to 0. This causes many spurious states 

In [ ]:
dims_in =  [2, 1, 2, 1, 5, 2,10, 3, 2, 1, 3, 2, 4, 2, 5,20,30,10,10,10,15]
dims_out = [1, 2, 1, 2, 5, 2, 7, 3, 2, 1, 5, 7, 2, 1, 2,20,30,10,10,10,15]
matrix_a = np.random.rand(sum(dims_out),2)@np.random.rand(2,sum(dims_in))
T = ToeplitzOperator(matrix_a, dims_in, dims_out)
S = SystemIdentificationSVD(T,epsilon=0)
system_a = MixedSystem(S)
print(system_a)

print("rank(matrix_a)=",np.linalg.matrix_rank(matrix_a))

In [ ]:
np.max(abs(matrix_a-system_a.to_matrix()))

Now we apply the reduction

In [ ]:
system_a_red = Reduction().apply(system_a)
print(system_a_red)
np.max(abs(matrix_a-system_a_red.to_matrix()))

We can see that the syste is still not minimal, even if the number of state dimentiosn has been reduced.
This is because the reduction mehto only takes the singualr values in a single step into account. 
Therefore we might sill not have a minimal system.
If we want to make the system minimal with an well defined croping of singular vaules of the Hankel operators we have to transform it to a ballanced realization. 

In this case we can use the reduce function with a bigger $\epsilon$

In [ ]:
system_a_red = Reduction(epsilon=1e-7).apply(system_a)
print(system_a_red)

In [ ]:
np.max(abs(matrix_a-system_a_red.to_matrix()))

We can also see that the reduction might introduced an error.

### Implementation of Output and Input normal

Some Notes here:

The input and output normal transformations use the QR-decomposition.

Here a short illustration of the output normal algorithm, the input normal algorithm is similar and it shoud be no problem to understand it if the output normal implementation is clear.

Output normality is $$A_k^\top A_k+C_k^\top C_k = 1$$

This is equivalent to 
$$
\begin{bmatrix}
   A_k^\top & C_k^\top \\
\end{bmatrix}
\begin{bmatrix}
   A_k\\
   C_k
\end{bmatrix}
=1
$$

To obtian this we want to make the matrix $\begin{bmatrix}
   A_k\\
   C_k
\end{bmatrix}$ orthogonal.

This can be done with state transforms of the structure

$$\hat{A}_k=S_{k+1} A_k S_k^{-1}$$
This can be done using the QR factorization. 
For this we look over the stages in descending order.
We already know the state transform $S_{k+1}$ form the previous step. (The inital step of the algorithm does not have a this state transform as the $A$ matrix has vanishing size)

If we apply the QR-factorization to the matrix $\begin{bmatrix}
   S_{k+1} A_k\\
   C_k
\end{bmatrix}$ we obtain the a $Q$ that can be split up as following

$$
\begin{bmatrix}
   S_{k+1} A_k\\
   C_k
\end{bmatrix} = QR = \begin{bmatrix}
   \hat{A}_k\\
   \hat{C}_k
\end{bmatrix} R
$$

We can now see that the matrices $\hat{A}_k$ and $\hat{A}_k$ fullfill the condition.
The $R$ is the transformation matrix $S_k$
Also watch out that the transformation with $S_k^{-1}$ is already implicitly done by the QR-decomposition.

As a last step we transform the matrices of the next stages with $S_k$

$$S_{k} A_{k-1}$$
$$S_{k} B_{k-1} $$

**Some notes on observability, reachability and minimality**

The algorithm produces a observable realization as the observability matrix $𝓞$ consists of orthogonal collumns.

Now the question is if the transformationalso preserves reachability.

For this we have to consider that the reachability matrix is is transformed according to:

$$ \hat{𝓡} = S_k 𝓡 $$

If $S_k$ has full rank and the original system is reachable the new observability matrix $\hat{𝓡}$ has also full rank and thus the system is observable.

The matrix $S_k$ has full rank iff the matries $\begin{bmatrix}
   A_k\\
   C_k
\end{bmatrix}$
have full rank. This is the case if the system is reachable.

Therefore the system preserves minimality.
If we would like to make the system minimal, we would need to use a rank revealing QR-decomposition or the SVD.
